In [1]:
import copy
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from PIL import Image
import torch.nn.functional as F
import matplotlib.pyplot as plt
from skimage import io, transform
from sklearn.metrics import confusion_matrix
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision
from torchvision import transforms, utils, models

data_dir = r'/media/nghia/DATA/DATA/Bee/bee_imgs/'
PATH = r'/media/nghia/DATA/DATA/Bee/bee_imgs/bee_data.csv'
img_path = r'/media/nghia/DATA/DATA/Bee/bee_imgs/bee_imgs'

df = pd.read_csv(PATH)

In [2]:
df['health'] = df['health'].map({'healthy': 0,
                                 'few varrao, hive beetles': 1,
                                 'Varroa, Small Hive Beetles': 2,
                                 'ant problems': 3,
                                 'hive being robbed': 4,
                                 'missing queen': 5})

transform = {'train': transforms.Compose([transforms.Resize(256),
                                          transforms.CenterCrop(224),
                                          #torchvision.transforms.ColorJitter(hue=.05, saturation=.05),
                                          #transforms.RandomHorizontalFlip(),
                                          transforms.ToTensor(),
                                          transforms.Normalize([0.485, 0.456, 0.406],
                                                               [0.229, 0.224, 0.225])]),

             'val': transforms.Compose([transforms.Resize(256),
                                        transforms.CenterCrop(224),
                                        transforms.ToTensor(),
                                        transforms.Normalize([0.485, 0.456, 0.406],
                                                             [0.229, 0.224, 0.225])]),

             'test': transforms.Compose([transforms.Resize(256),
                                         transforms.CenterCrop(224),
                                         transforms.ToTensor(),
                                         transforms.Normalize([0.485, 0.456, 0.406],
                                                              [0.229, 0.224, 0.225])])}

# Check for cuda
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
class HoneyBeeDataset(Dataset):
    # instance attributes
    def __init__(self, df, csv_file, root_dir, transform=None):
        self.data = df
        self.root_dir = root_dir
        self.labels = np.asarray(self.data.iloc[:, 6])
        self.transform = transform

    # length of the dataset passed to this class method
    def __len__(self):
        return len(self.data)

    # get the specific image and labels given the index
    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.data.iloc[idx, 0])
        image = Image.open(img_name)
        image = image.convert('RGB')
        image_label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, image_label

In [4]:
dataset = HoneyBeeDataset(df=df,
                          csv_file=PATH,
                          root_dir=img_path)

validation_split = 0.2
te_split = 0.5
dataset_size = len(dataset)
indices = list(range(dataset_size))
np.random.shuffle(indices)
val_split = int(np.floor(validation_split * dataset_size))
test_split = int(np.floor(te_split * val_split))
train_indices = indices[val_split:]
rest_indices = indices[:val_split]
val_indices, test_indices = rest_indices[test_split:], rest_indices[:test_split]

In [5]:
dataset_sizes = {'train': len(train_indices), 'val': len(val_indices), 'test': len(test_indices)}

train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)
test_sampler = SubsetRandomSampler(test_indices)

train_dataset = HoneyBeeDataset(df=df,
                                csv_file=PATH,
                                root_dir=img_path,
                                transform=transform['train'])

val_dataset = HoneyBeeDataset(df=df,
                              csv_file=PATH,
                              root_dir=img_path,
                              transform=transform['val'])

test_dataset = HoneyBeeDataset(df=df,
                               csv_file=PATH,
                               root_dir=img_path,
                               transform=transform['test'])

In [6]:
dataloaders = {'train': torch.utils.data.DataLoader(train_dataset, batch_size=4, sampler=train_sampler),
               'val': torch.utils.data.DataLoader(val_dataset, batch_size=4, sampler=valid_sampler),
               'test': torch.utils.data.DataLoader(test_dataset, batch_size=1, sampler=test_sampler)}

In [7]:
def train_model(model, criterion, optimizer, scheduler, num_epochs):
    #copy the best model weights
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print("Epoch: {}/{}".format(epoch, num_epochs-1))
        print("="*10)

        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for data in dataloaders[phase]:
                inputs, labels = data
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase=='train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase=='train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item()*inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

# LeNet

In [12]:
class LeNet(nn.Module):
    def __init__(self, output_dim):
        super().__init__()

        self.conv1 = nn.Conv2d(in_channels=3,
                               out_channels=6,
                               kernel_size=5)

        self.conv2 = nn.Conv2d(in_channels=6,
                               out_channels=16,
                               kernel_size=5)

        self.fc_1 = nn.Linear(44944, 16)
        self.fc_2 = nn.Linear(16, 16)
        self.fc_3 = nn.Linear(16, output_dim)

    def forward(self, x):

        # x = [batch size, 3, 100, 100]
        #print(x.shape)
        x = self.conv1(x)
        #print(x.shape)
        # x = [batch size, 6, 96, 96]

        x = F.max_pool2d(x, kernel_size=2)
        #print(x.shape)
        # x = [batch size, 6, 48, 48]

        x = F.relu(x)

        x = self.conv2(x)
        #print(x.shape)
        # x = [batch size, 6, 44, 44]

        x = F.max_pool2d(x, kernel_size=2)
        #print(x.shape)
        # x = [batch size, 6, 22, 22]

        x = F.relu(x)
        #print(x.shape)
        x = x.view(x.shape[0], -1)
        #print(x.shape)
        # x = [batch size, 16*4*4 = 256]

        h = x

        x = self.fc_1(x)

        # x = [batch size, 120]

        x = F.relu(x)

        x = self.fc_2(x)

        # x = batch size, 84]

        x = F.relu(x)

        x = self.fc_3(x)

        # x = [batch size, output dim]

        return x

OUTPUT_DIM = 6
model = LeNet(OUTPUT_DIM)
model

LeNet(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc_1): Linear(in_features=44944, out_features=16, bias=True)
  (fc_2): Linear(in_features=16, out_features=16, bias=True)
  (fc_3): Linear(in_features=16, out_features=6, bias=True)
)

In [13]:
model = model.to(device)
# loss function
criterion = nn.CrossEntropyLoss()
# Observe that all parameters are being optimized
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 10 epochs
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [14]:
EPOCHS = 35
# train
model_pre = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=EPOCHS)

Epoch: 0/34


/home/nghia/CNU-SEM3/AI_proj/Bee_Health/Eagle-Vision/streamlit/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


train Loss: 0.8334 Acc: 0.7078
val Loss: 0.4756 Acc: 0.8240
Epoch: 1/34
train Loss: 0.5223 Acc: 0.7997
val Loss: 0.4500 Acc: 0.8259
Epoch: 2/34
train Loss: 0.4353 Acc: 0.8260
val Loss: 0.3636 Acc: 0.8569
Epoch: 3/34
train Loss: 0.3613 Acc: 0.8424
val Loss: 0.2840 Acc: 0.8743
Epoch: 4/34
train Loss: 0.3391 Acc: 0.8519
val Loss: 0.2744 Acc: 0.8859
Epoch: 5/34
train Loss: 0.3108 Acc: 0.8591
val Loss: 0.4842 Acc: 0.8104
Epoch: 6/34
train Loss: 0.4266 Acc: 0.8306
val Loss: 0.3912 Acc: 0.8433
Epoch: 7/34
train Loss: 0.3962 Acc: 0.8461
val Loss: 0.3563 Acc: 0.8685
Epoch: 8/34
train Loss: 0.3579 Acc: 0.8574
val Loss: 0.3137 Acc: 0.8762
Epoch: 9/34
train Loss: 0.2509 Acc: 0.8835
val Loss: 0.2543 Acc: 0.8839
Epoch: 10/34
train Loss: 0.2286 Acc: 0.8862
val Loss: 0.2565 Acc: 0.8781
Epoch: 11/34
train Loss: 0.2161 Acc: 0.8925
val Loss: 0.2577 Acc: 0.8781
Epoch: 12/34
train Loss: 0.2068 Acc: 0.8942
val Loss: 0.2623 Acc: 0.9014
Epoch: 13/34
train Loss: 0.2005 Acc: 0.8937
val Loss: 0.2515 Acc: 0.8897


In [15]:
def test_model():
    running_correct = 0
    running_total = 0
    true_labels = []
    pred_labels = []
    # no gradient calculation
    with torch.no_grad():
        for data in dataloaders['test']:
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            true_labels.append(labels.item())
            outputs = model_pre(inputs)
            _, preds = torch.max(outputs.data, 1)
            pred_labels.append(preds.item())
            running_total += labels.size(0)
            running_correct += (preds == labels).sum().item()
    return (true_labels, pred_labels, running_correct, running_total)

In [16]:
true_labels, pred_labels, running_correct, running_total= test_model()
print('Correct: {}, Total: {}'.format(running_correct, running_total))
print('Test Accuracy: ', (running_correct/running_total))

Correct: 470, Total: 517
Test Accuracy:  0.9090909090909091


In [17]:
# clf report
from sklearn.metrics import classification_report
print(classification_report(true_labels, pred_labels))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98       352
           1       0.63      0.71      0.67        55
           2       0.69      0.53      0.60        45
           3       1.00      0.95      0.97        38
           4       0.91      0.84      0.87        25
           5       1.00      1.00      1.00         2

    accuracy                           0.91       517
   macro avg       0.87      0.84      0.85       517
weighted avg       0.91      0.91      0.91       517

